# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import glob
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
from py.quantstats_fix import *
from py.utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/__init__.py
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/core.py...
✓ Found 'plot_timeserie

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=10*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2015-06-06 to 2025-06-03
Time span: 3650 days (10.00 years)


#### Report File

In [4]:
# Get the most recent portfolio file
portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
if portfolio_files:
    # Sort files by modification time (most recent first)
    output_file = max(portfolio_files, key=os.path.getmtime)
    print(f"Using most recent portfolio file: {output_file}")
else:
    # Fallback to current date if no files found
    output_file = f'portfolios/portfolio-{datetime.date(end_date)}.xlsx'
    print(f"No portfolio files found. Using: {output_file}")

Using most recent portfolio file: portfolios/portfolio-2025-06-02.xlsx


#### Benchmark

In [5]:
# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark_long_term")['Benchmark'].values[0]
print(benchmark)

YYY


#### Risk-free rate (T-bill, %)

In [6]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.042300000000000004 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [7]:
portfolio_df = pd.read_excel(output_file, sheet_name="long_term")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['MMC', 'TMUS', 'APO', 'SO', 'AMAT']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W High,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,MMC,2025-06-02,"Marsh & McLennan Companies, Inc.",Financial Services,Insurance Brokers,United States,https://www.marshmclennan.com,115130826752,136148770816,491821141,...,248.00,205.17,229.54,225.30,2.62,0.01,0.337054,0.146168,0.203685,0.717616
1,TMUS,2025-06-02,"T-Mobile US, Inc.",Communication Services,Telecom Services,United States,https://www.t-mobile.com,275005997056,384693796864,452736028,...,276.49,171.18,251.20,232.97,3.11,0.03,0.337054,0.160654,0.265561,0.604961
2,APO,2025-06-02,"Apollo Global Management, Inc.",Financial Services,Asset Management,United States,https://www.apollo.com/institutional/homepage,74688552960,94705582080,446479870,...,189.49,95.11,133.28,143.86,7.75,0.05,0.161474,0.166024,0.375845,0.441734
3,SO,2025-06-02,The Southern Company,Utilities,Utilities - Regulated Electric,United States,https://www.southerncompany.com,99017097216,170091954176,1095713937,...,94.45,76.96,89.88,87.92,5.08,0.02,0.090374,0.097615,0.219272,0.445177
4,AMAT,2025-06-02,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,United States,https://www.appliedmaterials.com,125791559680,125714595840,799392549,...,255.89,123.74,151.52,173.39,2.58,0.02,0.074043,0.142597,0.413261,0.345053


### Step 4: Download Returns

In [8]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('data/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('data/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 5 of 5 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
AMAT: 0.5310
APO: 0.5971
MMC: 0.7723
SO: 0.5344
TMUS: 0.6942


,AMAT,APO,MMC,SO,TMUS
Date,,,,,
2015-06-09,0.001701,-0.034938,0.003888,-0.003577,-0.017769
2015-06-10,0.006776,0.002220,0.016007,-0.002511,-0.003169
2015-06-11,-0.013028,0.002215,0.008008,0.006088,-0.014117
2015-06-12,-0.008014,-0.005917,-0.003796,-0.006806,0.022807
2015-06-15,0.004587,-0.005206,-0.005017,-0.005045,-0.001049


### Step 5: Plot Return Comparisons

In [9]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
MMC: 33.71%
TMUS: 33.71%
APO: 16.15%
SO: 9.04%
AMAT: 7.40%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 0.9076
Sharpe Ratio (Equal-Weight): 0.8616


### Step 6: Generate Reports

#### Portfolio (Weighted) vs Benchmark

In [10]:
# portfolio_return = portfolio_return.resample('D').sum()  # Example of valid aggregation
portfolio_return.name = "Weighted Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    benchmark_returns,
    rf=risk_free_rate,
    figsize=(8, 5),
    # output=f'portfolio_vs_{benchmark}.html',
    output=f'portfolios/portfolio_vs_benchmark_long_term-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark)',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return, 
    benchmark_returns,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark}',
    benchmark_title=f'{benchmark}') 

/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_long_term-2025-06-03.html


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retai

                           YYY         Strategy
-------------------------  ----------  ----------
Start Period               2015-06-09  2015-06-09
End Period                 2025-05-30  2025-05-30
Risk-Free Rate             4.23%       4.23%
Time in Market             90.0%       100.0%

Cumulative Return          34.34%      386.3%
CAGR﹪                     0.01%       0.04%

Sharpe                     -0.01       0.7
Prob. Sharpe Ratio         11.98%      76.67%
Smart Sharpe               -0.01       0.62
Sortino                    -0.01       0.97
Smart Sortino              -0.01       0.87
Sortino/√2                 -0.01       0.69
Smart Sortino/√2           -0.0        0.61
Omega                      1.14        1.14

Max Drawdown               -45.0%      -37.19%
Longest DD Days            1425        369
Volatility (ann.)          14.48%      19.64%
R^2                        0.44        0.44
Information Ratio          0.06        0.06
Calmar                     0.05        0.

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2020-02-20,2020-03-23,2020-11-12,267,-37.193100,-32.509945
2,2018-01-29,2018-12-24,2019-02-01,369,-16.414639,-13.238017
3,2015-08-11,2016-02-11,2016-04-18,252,-16.362822,-15.434600
4,2021-12-28,2022-06-16,2022-08-11,227,-16.131369,-14.088521
5,2022-08-16,2022-10-14,2022-11-29,106,-15.951241,-15.477576


/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/home/renanmogo/mfin-algo-trading-team8/.venv/lib/python3.13/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/home/renanmogo/mfin-alg

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [11]:
# # Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
# equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
# portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# # Set the name for the equal weight portfolio
# equal_weight_return.name = "Equal Weight Portfolio"

# # Generate Report
# qs.reports.html(
#     portfolio_return,
#     equal_weight_return,
#     rf=risk_free_rate,
#     figsize=(8, 5),
#     output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
#     title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
#     benchmark_title="Equal Weight Portfolio",
#     download_filename="portfolio_vs_equal_weight.html" 
# )

# qs.reports.full(
#     portfolio_return, 
#     equal_weight_return,
#     rf=risk_free_rate, 
#     figsize=(8, 5), 
#     title='Portfolio vs Equal Weight',
#     benchmark_title="Equal Weight Portfolio",
#     ) 